In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
import tensorflow as tf
import warnings
import tensorflow_hub as hub
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
warnings.filterwarnings('ignore')
from sklearn.metrics import pairwise_distances
from heapq import nsmallest
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi

In [2]:
test_articles = pd.read_csv('../dataset/test_articles_newyork.csv', delimiter=',')

In [3]:
fixed_index = pd.read_csv('../dataset/fixed_test_set_newyork_max_recall5.csv', delimiter=',')

In [4]:
MAX_PAD = 30

In [5]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID[0] == ID_goal:
    
            accuracy = 1
            break;

    return accuracy

In [6]:
def get_mrr(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    index_match = 1

    for idTable in ranked_tables_ID:
        
        if idTable[0] == ID_goal:
    
            accuracy = 1/index_match
            break;
        
        index_match = index_match + 1
   
    return accuracy

In [7]:
test_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   article_full_text         141 non-null    object
 1   article_key_match         148 non-null    object
 2   article_keywords          143 non-null    object
 3   article_main_passage      148 non-null    object
 4   article_meta_description  132 non-null    object
 5   article_meta_keywords     58 non-null     object
 6   article_summary           141 non-null    object
 7   article_tags              54 non-null     object
 8   article_title             148 non-null    object
 9   article_url               148 non-null    object
dtypes: object(10)
memory usage: 11.7+ KB


In [8]:
fixed_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14800 entries, 0 to 14799
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   label_index              14800 non-null  object
 1   table_page_id            14800 non-null  object
 2   table_page_title         14800 non-null  object
 3   table_page_main_passage  14151 non-null  object
 4   table_page_keywords      10971 non-null  object
dtypes: object(5)
memory usage: 578.2+ KB


In [9]:
evaluate_topk = [1,5,10,20,50]

result = []

#evaluating each topk value
for TOP_K in evaluate_topk:

    accuracy = []
    mrr = []

    for i, row in tqdm(test_articles.iterrows()):

        #current article values
        article_ID = row['article_key_match']
        article_title_text = str(row['article_title'])
        article_title_text = ' '.join(article_title_text.split()[:MAX_PAD])
        

        #embedding and model variables
        table_corpus = []
        ranked_tables_model = []

        #return index
        return_index = fixed_index.loc[fixed_index['label_index'] == row['article_key_match']]
        

        #creating embedding 
        for i, row in return_index.iterrows():
            
            table_page_title_text = ' '.join(str(row['table_page_title']).split()[:MAX_PAD])
            
            table_corpus.append(table_page_title_text)
              
        tokenized_corpus = [doc.split(" ") for doc in table_corpus]
        bm25 = BM25Okapi(tokenized_corpus)
        
        tokenized_query = article_title_text.split(" ")
        distance_vector = bm25.get_scores(tokenized_query)
        
        #creating the final dataframe
        for i in range(0,len(distance_vector)):

            ranked_tables_model.append([return_index.iloc[i]['table_page_id'],return_index.iloc[i]['table_page_title'],distance_vector[i]]) 

        data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
        data_frame_sorting = data_frame.sort_values('table_ranking', ascending=False)
        
        selected_top = data_frame_sorting.head(TOP_K)
#         min_score = selected_top['table_ranking'].min()
#         draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] >= min_score]
        final_ranked_tables = selected_top.iloc[:,0:1].values

        #getting topk accuracy
        accuracy.append(get_accuracy(article_ID, final_ranked_tables))

        #testing mean reciprocal rank at k = 50
        if TOP_K == 50:

            mrr.append(get_mrr(article_ID, final_ranked_tables))

    result.append(["Acc@"+str(TOP_K),str(round(np.mean(accuracy),4))])

print("BM25 METHOD")
print(result[0])
print(result[1])
print(result[2])
print(result[3])
print(result[4])
print("MRR: "+str(round(np.mean(mrr),4)) )
print(mrr)
print("")

148it [00:07, 19.78it/s]
148it [00:06, 22.08it/s]
148it [00:06, 22.51it/s]
148it [00:07, 19.71it/s]
148it [00:07, 19.36it/s]

BM25 METHOD
['Acc@1', '0.3108']
['Acc@5', '0.4932']
['Acc@10', '0.6216']
['Acc@20', '0.7095']
['Acc@50', '0.7838']
MRR: 0.4085
[0, 0, 0.5, 0.5, 0.041666666666666664, 0, 0.16666666666666666, 0, 1.0, 0.5, 0.05, 0, 0.2, 0.2, 0.08333333333333333, 0.07142857142857142, 0, 1.0, 1.0, 1.0, 0, 0, 0.16666666666666666, 0.16666666666666666, 1.0, 0, 0, 1.0, 1.0, 0.125, 0.029411764705882353, 0.0625, 0.125, 0.041666666666666664, 0.16666666666666666, 0.02, 1.0, 0.0625, 0, 1.0, 0.1111111111111111, 0.1, 0.25, 0.3333333333333333, 1.0, 0, 1.0, 0, 0, 1.0, 0.125, 0.3333333333333333, 1.0, 0.04, 1.0, 0.25, 1.0, 0, 1.0, 1.0, 0.16666666666666666, 0.029411764705882353, 0.0625, 0.2, 1.0, 0.2, 0, 1.0, 1.0, 0.3333333333333333, 1.0, 1.0, 0.5, 0.07692307692307693, 0, 0.16666666666666666, 1.0, 0.25, 0.5, 0, 0.14285714285714285, 0.5, 1.0, 0.0625, 0.5, 1.0, 1.0, 1.0, 0.08333333333333333, 0, 1.0, 1.0, 1.0, 0, 0, 0.08333333333333333, 0.125, 0.02564102564102564, 0.1111111111111111, 0.1, 0.5, 1.0, 0, 0, 0, 0, 1.0, 1.0, 0, 0.